Import request package that can communicate with web

'Get' function to access info

'Post' function to modify html info

Import BeautifulSoup package that can parse html language

Import regular expression package to parse sentence

A successful retrieval should return response status code '200'

https://www.restapitutorial.com/httpstatuscodes.html


In [ ]:
from bs4 import BeautifulSoup
import requests
import re

site_id="https://www.tripadvisor.co.uk/Restaurant_Review-g503775-d2519242-Reviews-The_Gurkha_Restaurant-Chelmsford_Essex_England.html"
response = requests.get(site_id)
print(response.status_code)
html = BeautifulSoup(response.text, 'html.parser')

#### Determine the html tags by searching with sentence

In [ ]:
header_parent = html.find(text='Never Fails to Impress').parent
content_parent = html.find(text=re.compile(r"Great service, great food, our favourite restaurant in Chelmsford. The welcome from the host is second to none, great atmosphere. Will be back soon.")).parent#.parent.parent
reply_parent = html.find(text=re.compile(r"Thank you for taking the time")).parent.parent.parent.parent
print(header_parent) # Get parents for the text
print("===================")
print(content_parent)
print("===================")
print(reply_parent)

In [ ]:
reviews_no = html.find('span', class_='reviews_header_count').text[1:-1] # return (1,XXX), remove () in string
reviews_no = reviews_no.replace(',', '') # remove `,`
reviews_no = int(reviews_no)
print(reviews_no)

In [ ]:
reviews_header=list()
for header in html.find_all('span', class_='noQuotes'):
    reviews_header.append(header.text)
    
for element in reviews_header:
    print(element)
    print("===================")

In [ ]:
reviews_content=list()

for content in html.findAll('div', class_='review-container'):
    if content.find(class_='mgrRspnInline'): # take out manager's reply
        continue
    elif content.find('p', class_='partial_entry'):
        append_content = content.find('p', class_='partial_entry')
        reviews_content.append(append_content.text)
        

for element in reviews_content:
    print(element)
    print("===================")

In [ ]:
reviews_header=list()
reviews_rating=list()
reviews_content=list()

for review in html.find_all('div', class_='review-container'):
    header=review.find('span', class_='noQuotes').text
    reviews_header.append(header)
    
    if review.find(class_='mgrRspnInline'): # take out manager's reply
        continue
    elif review.find('p', class_='partial_entry'):
        append_content = review.find('p', class_='partial_entry').text
        reviews_content.append(append_content)
        
    if review.find(class_='bubble_50'):
        rating='5'
    elif review.find(class_='bubble_40'):
        rating='4'
    elif review.find(class_='bubble_30'):
        rating='3'
    elif review.find(class_='bubble_20'):
        rating='2'
    elif review.find(class_='bubble_10'):
        rating='1'
    reviews_rating.append(rating)
    

for element1, element2, element3 in zip(reviews_header, reviews_content, reviews_rating):
    print(element1)
    print(element2)
    print(element3)
    print("===================")


In [ ]:
from bs4 import BeautifulSoup
import requests
import re

results=list()
no_response=False

def get_url(current_url):
    file = requests.get(current_url)
    if file.status_code!=200:
        print('no response')
        no_respnse=True
    
    response = BeautifulSoup(file.text, 'html.parser')
    return [response, no_response]

def get_reviews(url, response):
    print(url)
    # get every review
    for review in response.find_all('div', class_='review-container'):
        
        if review.find(class_='mgrRspnInline'): # take out manager's reply
            continue
        elif review.find('p', class_='partial_entry'):
            append_content = review.find('p', class_='partial_entry').text
            
            if review.find(class_='bubble_50'):
                rating='5'
            elif review.find(class_='bubble_40'):
                rating='4'
            elif review.find(class_='bubble_30'):
                rating='3'
            elif review.find(class_='bubble_20'):
                rating='2'
            elif review.find(class_='bubble_10'):
                rating='1'
            
        item = {
            'Restaurant_name': response.find('h1', class_='ui_header h1').text, # Restaurant name is outside of review-container
            'review_title': review.find('span', class_='noQuotes').text,
            'review_content': append_content,
            'review_rating': rating
        }

        results.append(item) # <--- add to global list

        #~ yield item
        for key,val in item.items():
            print(key, ':', val)
        print('----')
        # return for test only - to stop after first review
        

In [ ]:
for idx in range(0,109):
    s='' if idx==0 else '-or{0}0'.format(idx)
    site_id = "https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d5244301-Reviews{0}-The_Golden_Chippy-London_England.html".format(s)
    [response, no_response] = get_url(site_id)
    if no_response == False:
        get_reviews(site_id, response)
    else:
        break
    
import pandas as pd

df = pd.DataFrame(results) # <--- convert list to DataFrame  

In [ ]:
df.to_csv('{0}.csv'.format(df['Restaurant_name'][0]))  